In [ ]:
import zarr
import numpy as np
import xarray as xr
import torch

In [ ]:
bs = 1
autoregression_steps = 1
timesteps_cnt = 24
levels_cnt = 5
vars_cnt = 5
lats_cnt = 121
lons_cnt = 240

In [ ]:
class Model:
    def __init__(self):
        pass

    def forward(self, x):
        return torch.rand((bs, autoregression_steps, levels_cnt * vars_cnt, lats_cnt, lons_cnt))


model = Model()

In [ ]:
store = zarr.DirectoryStore('./preds.zarr')
root = zarr.group(store=store, overwrite=True)

latitude = root.create_dataset('lats', shape=(0,), chunks=(lats_cnt,), dtype=np.float64, fill_value=0)
levels = root.create_dataset('levels', shape=(0,), chunks=(levels_cnt,), dtype=np.int32, fill_value=0)
longitude = root.create_dataset('lons', shape=(0,), chunks=(lons_cnt,), dtype=np.float64, fill_value=0)
# pred_timedelta muss noch implementiert werden, ich glaub im zusammenhang mit autoregression steps
# pred_timedelta = root.create_dataset('pred_timedelta', shape=(10,), dtype='timedelta64[ns]')
# die Zeit muss noch richtig gesetzt werden, wahrscheinlich dann über das Dataset
time = root.create_dataset('time', shape=(0,), chunks=(timesteps_cnt,), dtype='datetime64[ns]', fill_value=0)

temp = root.create_dataset('temperature', shape=(0, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                           chunks=(16, levels_cnt, lats_cnt, lons_cnt), fill_value=0)
temp.attrs['_ARRAY_DIMENSIONS'] = ['time', 'levels', 'lats', 'lons']
humid = root.create_dataset('specific_humidity', shape=(0, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                            chunks=(16, levels_cnt, lats_cnt, lons_cnt), fill_value=0)
humid.attrs['_ARRAY_DIMENSIONS'] = ['time', 'levels', 'lats', 'lons']
uwind = root.create_dataset('u_component_of_wind', shape=(0, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                            chunks=(16, levels_cnt, lats_cnt, lons_cnt), fill_value=0)
uwind.attrs['_ARRAY_DIMENSIONS'] = ['time', 'levels', 'lats', 'lons']
vwind = root.create_dataset('v_component_of_wind', shape=(0, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                            chunks=(16, levels_cnt, lats_cnt, lons_cnt), fill_value=0)
vwind.attrs['_ARRAY_DIMENSIONS'] = ['time', 'levels', 'lats', 'lons']
geo = root.create_dataset('geopotential', shape=(0, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                          chunks=(16, levels_cnt, lats_cnt, lons_cnt), fill_value=0)
geo.attrs['_ARRAY_DIMENSIONS'] = ['time', 'levels', 'lats', 'lons']


In [ ]:
latitude.append(np.linspace(-90, 90, lats_cnt))
latitude.attrs['_ARRAY_DIMENSIONS'] = ['lats']

levels.append(np.arange(levels_cnt))
levels.attrs['_ARRAY_DIMENSIONS'] = ['levels']

longitude.append(np.linspace(-180, 180, lons_cnt))
longitude.attrs['_ARRAY_DIMENSIONS'] = ['lons']

time.append(np.arange(timesteps_cnt))
time.attrs['_ARRAY_DIMENSIONS'] = ['time']

In [ ]:
# range muss noch durch den dataloader ersetzt werden
for idx, batch in enumerate(range(timesteps_cnt)):
    # model muss noch durch ein richtiges Modell ersetzt werden
    # die 0 steht für die autoregression steps
    preds = torch.reshape(model.forward(batch)[0, 0, :, :, :], (vars_cnt, levels_cnt, lats_cnt, lons_cnt)).numpy()
    # Hier muss noch sinnvoll die Autoregression Steps implementiert werden
    # Da bin ich mir aber gerade nicht sicher, wie die richtig rein müssen
    # wahrscheinlich übers timedelta
    temp.append(preds[None, 0, :, :, :], axis=0)
    humid.append(preds[None, 1, :, :, :], axis=0)
    uwind.append(preds[None, 2, :, :, :], axis=0)
    vwind.append(preds[None, 3, :, :, :], axis=0)
    geo.append(preds[None, 4, :, :, :], axis=0)

In [ ]:
zarr.consolidate_metadata(store)

In [ ]:
# beim laden mit xarray kommt es zum problem, dass 0´en durch nan ersetzt werden
b = xr.open_zarr('./preds.zarr')
b